In [1]:
import json
import torch
import numpy as np
import os
from torch.utils.data import Dataset, DataLoader, random_split
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader as PyGDataLoader
import torch.nn as nn
from torch_geometric.nn import GCNConv, global_mean_pool
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sys
from torch_geometric.utils import to_dense_batch
import torch.optim as optim
import json

sys.path.append(os.path.abspath(".."))

## Phase 1: Generating the Graphs

#### 1. Download Data

- To download only one episode, use data_downloader(ep_num,...)

- To download a range of episodes, use data_downloader(min_ep,max_ep,...)

- To download a list of episodes, use data_downloader(list,...)

- n_secs defines the interval of times between frames (default is set to 3 seconds)

- Additionally, you can chnge the following directories:

    - tabular_data_dir = '../data/raw/L2D/tabular'

    - frames_dir = '../data/raw/L2D/frames'

    - metadata_dir = '../data/raw/L2D/metadata'

In [2]:
from functions.load_data_L2D import data_downloader

data_downloader(0,n_secs=3,
                features={"tabular": True,
                          "frames": {
                                'observation.images.front_left': True,
                                'observation.images.left_backward': False,
                                'observation.images.left_forward': False,
                                'observation.images.map': False,
                                'observation.images.rear': False,
                                'observation.images.right_backward': False,
                                'observation.images.right_forward': False,
                            }
                        })

Using cached metadata at ../data/raw/L2D/metadata/metadata.parquet


videos/observation.images.front_left/chu(…):   0%|          | 0.00/482M [00:00<?, ?B/s]

✅ All done. Temporary files cleaned up.


#### 2. Process Tabular Data & Add Tags

- To process only one episode, use process_tabular_data(ep_num,...)

- To process a range of episodes, use process_tabular_data(min_ep,max_ep,...)

- To process a list of episodes, use process_tabular_data(list,...)

- Additionally, you can change the following directories:

    - process_tabular_data:

        - source_dir = '../data/raw/L2D/tabular'

        - output_dir_processed = '../data/processed_data/L2D'

        - output_dir_tags = '../data/semantic_tags/L2D'

    - add_data_tags:

        - data_dir = '../data/processed/L2D'

        - tags_dir='../data/semantic_tags/L2D'

In [2]:
from functions.process_tabular_data_L2D import process_tabular_data

process_tabular_data(0,
                    overwrite=True, process_columns=True, 
                    process_osm=True, process_turning=True,
                    time_sleep=2)

100%|██████████| 1/1 [00:34<00:00, 34.01s/it, errors=3 / 7]


In [3]:
from functions.process_tags_L2D import add_data_tags

add_data_tags(0)

Generating semantic tags: 100%|██████████| 1/1 [00:00<00:00, 59.04it/s]


#### 3. Process Frames

- Set up with depth model with pip install git+https://github.com/apple/ml-depth-pro.git

- Set up rfdetr with pip install --ignore-installed rfdetr

- To process only one episode, use process_frames(ep_num,...)

- To process a range of episodes, use process_frames(min_ep,max_ep,...)

- To process a list of episodes, use process_frames(list,...)

- Additionally, you can change the following directories:

    - input_base_dir = '../data/raw/L2D/frames',
    
    - output_base_dir = '../data/processed_frames/L2D'

- You can also include the following cameras:

    - "observation.images.front_left",
    - "observation.images.left_forward", 
    - "observation.images.right_forward",
    - "observation.images.right_backward",
    - "observation.images.rear",
    - "observation.images.left_backward"

**Note**: Including additional cameras will help with the depth, speed, etc. but CONSIDERABLY increases running time.

In [4]:
from functions.process_frames_L2D import process_frames

process_frames(0,
               cameras_on=["observation.images.front_left"],
               run_dict={"detection": True,
                         "depth": True,
                         "speed": True,
                         'overwrite': True},
                verbose=True)

Loading pretrain weights


100%|██████████| 1/1 [07:54<00:00, 474.58s/it]


In [5]:
from functions.process_lanes_L2D import lane_processing
summary = lane_processing(0)

100%|██████████| 1/1 [00:00<00:00,  2.89it/s]


#### 4. Generate Graphs (Need to Update for the Addition of Speed)

- To process only one episode, use generate_graphs(ep_num,...)

- To process a range of episodes, use generate_graphs(min_ep,max_ep,...)

- To process a list of episodes, use generate_graphs(list,...)

- Additionally, you can change the following directories:

    - source_data_dir = '../data/processed/L2D',
    
    - processed_frame_dir = '../data/processed_frames/L2D',
                    
    - output_dir = '../data/graphical/L2D'

In [2]:
from functions.graphs import generate_graphs
generate_graphs(0)

100%|██████████| 1/1 [00:00<00:00, 12.45it/s]


#### Quick Visualization of Graph & Tags

In [3]:
from functions.graphs import combined_graph_viewer

ep_num = 0
with open(f"../data/graphical/l2d/{ep_num}_graph.json", "r") as f:
    graph_data = json.load(f)

cyto = combined_graph_viewer({'graph':graph_data})
display(cyto)

CytoscapeWidget(cytoscape_layout={'name': 'cola'}, cytoscape_style=[{'selector': 'node', 'style': {'label': 'd…

In [5]:
ep_num = 0
with open(f"../data/semantic_tags/l2d/episode_{ep_num:06}.json", "r") as f:
    graph_tags = json.load(f)
graph_tags

{'episode_index': 0,
 'action_tag': 'straight',
 'traffic_control_tag': 'unmarked',
 'road_feature_tags': [],
 'environment_tags': ['clouds',
  'day',
  'low_visibility_possible',
  'off_peak_hours',
  'weekend',
  'winter_conditions_possible']}

In [6]:
from functions.graphs import graph_to_dfs

nodes_df, edges_df = graph_to_dfs(graph_data)
nodes_df.iloc[8].dropna()

id                                                            Veh_FL_A_1
type                                                             vehicle
lane_classification                                              in_lane
lane_overlap_ratio                                              0.408845
distance_m                                                      2.072108
dist_to_ego                                                      2.39308
speed_ms                                                        2.736617
speed_kmh                                                        9.85182
velocity_ms            [-0.3257415254237288, 2.7171610169491527, 8.03...
velocity_kmh           [-1.1726694915254237, 9.78177966101695, 28.914...
Name: 8, dtype: object